In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from cifar10_models import return_model

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:496: FutureWarning: Passing (type, 1) or 

## Select Model

In [2]:
name = 'resnet'
# New - Josue wants low/high alpha test. 
name = 'simplea3' # was simplehigha
name = 'simpleapoint33' # was simplelowa
name = 'simplea10'
name = 'simpleapoint1'
# New - Train Linear Models
name = 'linear' # actually linear
name = 'simplelinear' # conv linear
# New - Train nonlinear models
name = 'fclinear' # FC relu
name = 'lc' # LC Relu
name = 'circular' # FC Conv
name = 'simple' # LC Conv
# New - Ankit wants simplelinear with a = .1, 10
name = 'simplelineara10'
name = 'simplelinearapoint1'
# And now also for lc
name = 'lca10'
name = 'lcapoint1'
# And finally, lclinear
name = 'lclinear'
name = 'lclineara10'
name = 'lclinearapoint1'
# Add new circularlinear (which is the NONlinear circular conv)
name = 'circularlinear'

## Train Model (Saving every 30 iterations)

In [41]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os


#parser = argparse.ArgumentParser(description='PyTorch CIFAR10 Training')
#parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
#arser.add_argument('--resume', '-r', action='store_true',
#                    help='resume from checkpoint')
#args = parser.parse_args()



# For contaminated data, Josue wants linear vs nonlinear, FC vs Conv
# Linear FC : 'linear'
# Nonlinear FC : 'fclinear'
# Linear Conv : 'simplelinear'
# Nonlinear Conv : 'simple'

name = 'linear'
name = 'fclinear'
name = 'simplelinear'
name = 'simple'


args = {}
args['resume'] = False
args['lr'] = 0.1
#args['lr']/=10 # high alpha requires lower learning rate - try dividing by alpha to start, dividing by 3 gives 55.4% final test acc
# Basee lr works fine for linear, gets ~40% accuracy
# simple linear requires lr lower, /10 works ok to ~40% acc
args['lr']/=50
# fclinear also does well with /10 ~ 54+%
# lc also does well with /10, 59+%
# circular fails at /10 and /20, try /50, gets to around 41%
# final is simple, /50 seems to work fine, gets to around 57.5%
# simpelinear a = 10 got to around 40%
# '', a = .1 got to around 42%
# lc, a = 10, /10 got to around 56%
# lc, a = .1, /10 gets to around 54%
# lclinear (base) /10 got to 35%
# lclinear a = 10 /10 got to 30%
# lclinear,a = .1 /10 got to 41%
# circularlinear, /50 got to 39%

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
net = return_model(name)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

if args['resume']:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/ckpt.pth')
    net.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=args['lr'],
                      momentum=0.9, weight_decay=5e-4)


lambda1 = lambda epoch: 0.3**((epoch%20)//4)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda1])

def train(epoch,name):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        #if epoch == 0 and batch_idx == 0:
        if np.mod(batch_idx,30)==1:
            torch.save(net.state_dict(), './cifar_resnet/ckpt_{}_{}_{}.pth'.format(name, epoch, batch_idx))
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        #if batch_idx%1 == 0:
            #torch.save(net.state_dict(), './cifar_resnet/ckpt_track_{}_{}.pth'.format(epoch, batch_idx+1))
        if batch_idx%100 == 0:
            print(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'%(train_loss/(batch_idx+1), 100.*correct/total, correct, total))


def test(epoch,name):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

        print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'% (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('cifar_resnet'):
            os.mkdir('cifar_resnet')
        torch.save(state, './cifar_resnet/ckpt_{}.pth'.format(name))
        best_acc = acc


for epoch in range(start_epoch, start_epoch+40):
    train(epoch,name)
    test(epoch,name)
    scheduler.step()
print(best_acc)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building model..

Epoch: 0
0 391 Loss: 2.294 | Acc: 11.719% (15/128)
100 391 Loss: 2.055 | Acc: 25.201% (3258/12928)
200 391 Loss: 1.960 | Acc: 30.212% (7773/25728)
300 391 Loss: 1.906 | Acc: 32.641% (12576/38528)
99 100 Loss: 1.738 | Acc: 39.560% (3956/10000)
Saving..

Epoch: 1
0 391 Loss: 1.745 | Acc: 44.531% (57/128)
100 391 Loss: 1.712 | Acc: 40.524% (5239/12928)
200 391 Loss: 1.707 | Acc: 40.948% (10535/25728)
300 391 Loss: 1.693 | Acc: 41.380% (15943/38528)
99 100 Loss: 1.631 | Acc: 43.350% (4335/10000)
Saving..

Epoch: 2
0 391 Loss: 1.617 | Acc: 44.531% (57/128)
100 391 Loss: 1.613 | Acc: 45.204% (5844/12928)
200 391 Loss: 1.594 | Acc: 45.398% (11680/25728)
300 391 Loss: 1.581 | Acc: 45.590% (17565/38528)
99 100 Loss: 1.520 | Acc: 46.430% (4643/10000)
Saving..

Epoch: 3
0 391 Loss: 1.433 | Acc: 49.219% (63/128)
100 391 Loss: 1.482 | Acc: 48.886% (6320/12928)
200 391 Loss: 1.476 |

In [ ]:
#Non-poisoned: 
#L FC : 40.72, 40.75, 40.78, 40.83, 40.92
#NL FC: 48.63, 48.38, 48.53, 48.52, 48.45
#L Conv: 41.82, 41.87, 41.74, 41.77, 41.83
#NL Conv: 56.67, 55.84, 57.23, 57.07, 56.49

In [67]:
# New Experiment - try changing the dataset to have fourier features with uniform power at all frequencies
# Start by trying to load and modify a single image?

# ALl set up - now saves a new dataset based on 3 hyperparams epislon, p, rho
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from cifar10_models import return_model
import torchvision
import torchvision.transforms as transforms
import os

import matplotlib.pylab as plt
import numpy as np
from scipy.sparse import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.ToTensor(),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
])


trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=100, shuffle=False, num_workers=1) # no shuffle here, changed batch to 100

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=1)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

def get_fourier(imag):
    image_sample = np.fft.fft2(imag)
    F_orig = np.fft.fftshift(image_sample)
    #F_orig = np.log(np.abs(F_orig))
    return F_orig

new_train = np.zeros((50000,3,32,32))
new_train_classes = np.zeros(50000)
# Try setting up a system to generate a (class depedent) mask
epsilon = 2e-3 * 50 # random perturbation size - Ankit requested that it be 'comparable' to L1 norm of perturbations. These tend to be around .03, so choose .03/(32*32*p)? ~= 3e-5
p = .15 # random perturbation density from 1 for full to e.g. .01 for very sparse
rho = 0 # randomness applied to each individual image - e.g. making it non-deterministic
perturbation = np.random.randn(10,3,32,32)*epsilon
perturbation *= (np.random.rand(10,3,32,32)<p)
# just add perturbation[class,:,:] to each (fourier) data

print(np.linalg.norm(perturbation.flatten(),ord = 1))

# Load image
for batch_idx, (inputs, targets) in enumerate(trainloader):
    #print(inputs.shape) # 128 (batch) by 3 channels by 32 by 32
    #fig = plt.figure(figsize=(8, 8))
    #plt.imshow(inputs[0,0,:,:], vmin=None, vmax = None, interpolation='nearest')
    #plt.show()
    # Take fourier
    #ftest = get_fourier(inputs[0,0,:,:].data.numpy())
    #fig = plt.figure(figsize=(8, 8))
    #plt.imshow(np.abs(ftest), vmin=None, vmax = None, interpolation='nearest')
    #plt.show()
    # Try inverse?
    #ftest2 = np.fft.ifftshift(ftest)
    #forig = np.fft.ifft2(ftest2)
    #fig = plt.figure(figsize=(8, 8))
    #plt.imshow(np.real(forig), vmin=None, vmax = None, interpolation='nearest')
    #plt.show()
    # Back to original (more or less)
    
    # Test works - on to making the change
    for i in range(100): 
        fd1 = get_fourier(inputs[i,0,:,:].data.numpy())
        fd2 = get_fourier(inputs[i,1,:,:].data.numpy())
        fd3 = get_fourier(inputs[i,2,:,:].data.numpy())
        fd1 += perturbation[targets.numpy()[i],0,:,:]
        fd1 += np.random.randn(32,32)*rho
        fd2 += perturbation[targets.numpy()[i],1,:,:]
        fd2 += np.random.randn(32,32)*rho
        fd3 += perturbation[targets.numpy()[i],2,:,:]
        fd3 += np.random.randn(32,32)*rho
        new_train[100*batch_idx + i,0,:,:] = np.fft.ifft2(np.fft.ifftshift(fd1))
        new_train[100*batch_idx + i,1,:,:] = np.fft.ifft2(np.fft.ifftshift(fd2))
        new_train[100*batch_idx + i,2,:,:] = np.fft.ifft2(np.fft.ifftshift(fd3))

        new_train_classes[100*batch_idx + i] = targets.numpy()[i]
    
    # Ok works, now need to store the whole thing as a giant numpy array of size [samples] x [image,category]...
#np.save('./data/modified_cifar_train.npy', new_train)
np.savez('./data/modified_cifar_train_eps:{}_p:{}_rho:{}.npy'.format(epsilon, p, rho), images=new_train, target=new_train_classes)
# Have to do test set with same perturbation, so do that now too

new_test = np.zeros((10000,3,32,32))
new_test_classes = np.zeros(10000)
for batch_idx, (inputs, targets) in enumerate(testloader):
    for i in range(100): 
        fd1 = get_fourier(inputs[i,0,:,:].data.numpy())
        fd2 = get_fourier(inputs[i,1,:,:].data.numpy())
        fd3 = get_fourier(inputs[i,2,:,:].data.numpy())
        fd1 += perturbation[targets.numpy()[i],0,:,:]
        fd1 += np.random.randn(32,32)*rho
        fd2 += perturbation[targets.numpy()[i],1,:,:]
        fd2 += np.random.randn(32,32)*rho
        fd3 += perturbation[targets.numpy()[i],2,:,:]
        fd3 += np.random.randn(32,32)*rho
        new_test[100*batch_idx + i,0,:,:] = np.fft.ifft2(np.fft.ifftshift(fd1))
        new_test[100*batch_idx + i,1,:,:] = np.fft.ifft2(np.fft.ifftshift(fd2))
        new_test[100*batch_idx + i,2,:,:] = np.fft.ifft2(np.fft.ifftshift(fd3))

        new_test_classes[100*batch_idx + i] = targets.numpy()[i]
np.savez('./data/modified_cifar_test_eps:{}_p:{}_rho:{}.npy'.format(epsilon, p, rho), images=new_test, target=new_test_classes)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
375.19301904197795


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:92: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:93: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:94: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:116: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:117: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:118: ComplexWarning: Casting complex values to real discards the imaginary part


In [92]:
# Now that we have a dataset, try loading it and doing standard training
# Uses a new dataset, requires knowing the epsilon, p, rho that were used to produce it (these are in the file name)
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

from cifar10_models import return_model

import torchvision
import torchvision.transforms as transforms

import os

from numpy import load
import numpy as np

name = 'fclinearl1'
name = 'convlinearl1k32c3'

#name = 'convlinearl1k3c3'
#name = 'convlinearl1k3c32'

# For contaminated data, Josue wants linear vs nonlinear, FC vs Conv
# Linear FC : 'linear'
# Nonlinear FC : 'fclinear'
# Linear Conv : 'simplelinear'
# Nonlinear Conv : 'simple'

name = 'linear'
name = 'fclinear'
name = 'simplelinear'
name = 'simple'

args = {}
args['resume'] = False
args['lr'] = 0.1
args['lr']/=50
# .1 too fast for fclinearl1
# fclinearl1 /10 : 41.5%
# convlinearl1k32c3 /50 42%
# fclinearl1 v2 /10 42%
# convlinearl1k32c3 v2 /50 42%


device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')

class CIFAR10Dataset(torch.utils.data.Dataset):
    """CIFAR Dataset."""
    def __init__(self, dataset_path='./data/modified_cifar_test_eps:{}_p:{}_rho:{}.npy.npz'.format(3e-5,1,0), transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.file = np.load(dataset_path, allow_pickle=True)
        self.images = self.file['images'].astype(np.single)
        self.target = self.file['target']
        self.transform = transform
    def __len__(self):
        return len(self.target)
    def __getitem__(self, idx):
        img = self.images[idx]
        target = self.target[idx]
        if self.transform !=None:
            img = self.transform(img)
        img = torch.from_numpy(img)
        return img, target

transform_train = transforms.Compose([
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

epsilon = 5e-3 * 50 # random perturbation size - Ankit requested that it be 'comparable' to L1 norm of perturbations. These tend to be around .03, so choose .03/(32*32*p)? ~= 3e-5
p = .2 # random perturbation density from 1 for full to e.g. .01 for very sparse
rho = 0

#trainset = torchvision.datasets.CIFAR10(
#    root='./data', train=True, download=True, transform=transform_train)
#trainset = CIFAR10Dataset(dataset_path = './data/modified_cifar_train_eps:{}_p:{}_rho:{}.npy.npz'.format(3e-5,1,0))#, transform = transform_train)
#trainset = CIFAR10Dataset(dataset_path = './data/modified_cifar_train_eps:{}_p:{}_rho:{}.npy.npz'.format(0.0015,0.02,0))#, transform = transform_train)
trainset = CIFAR10Dataset(dataset_path = './data/modified_cifar_train_eps:{}_p:{}_rho:{}.npy.npz'.format(epsilon,p,rho))#, transform = transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=1)

#testset = torchvision.datasets.CIFAR10(
#    root='./data', train=False, download=True, transform=transform_test)
#testset = CIFAR10Dataset(dataset_path = './data/modified_cifar_test_eps:{}_p:{}_rho:{}.npy.npz'.format(3e-5,1,0))#, transform = transform_test)
#testset = CIFAR10Dataset(dataset_path = './data/modified_cifar_test_eps:{}_p:{}_rho:{}.npy.npz'.format(0.0015,0.02,0))#, transform = transform_test)
testset = CIFAR10Dataset(dataset_path = './data/modified_cifar_test_eps:{}_p:{}_rho:{}.npy.npz'.format(epsilon,p,rho))#, transform = transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=1)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')


#print(trainset.__getitem__(0))


# Model
print('==> Building model..')
net = return_model(name)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

if args['resume']:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/ckpt.pth')
    net.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=args['lr'],
                      momentum=0.9, weight_decay=5e-4)

# Change name - based on contamination
# Base (p = 1) - no change
# Sparse (p = .02) : v2
# Less sparse, stronger : v3

name +='v3'


lambda1 = lambda epoch: 0.3**((epoch%20)//4)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda1])

def train(epoch,name):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        targets = targets.type(torch.long)
        #print(inputs.shape)
        #if epoch == 0 and batch_idx == 0:
        if np.mod(batch_idx,30)==1:
            torch.save(net.state_dict(), './cifar_resnet/ckpt_{}_{}_{}.pth'.format(name, epoch, batch_idx))
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        #if batch_idx%1 == 0:
            #torch.save(net.state_dict(), './cifar_resnet/ckpt_track_{}_{}.pth'.format(epoch, batch_idx+1))
        if batch_idx%100 == 0:
            print(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'%(train_loss/(batch_idx+1), 100.*correct/total, correct, total))


def test(epoch,name):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            targets = targets.type(torch.long)
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

        print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'% (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('cifar_resnet'):
            os.mkdir('cifar_resnet')
        torch.save(state, './cifar_resnet/ckpt_{}.pth'.format(name))
        best_acc = acc


for epoch in range(start_epoch, start_epoch+40):
    train(epoch,name)
    test(epoch,name)
    scheduler.step()
print(best_acc)

==> Preparing data..
==> Building model..

Epoch: 0
0 391 Loss: 2.334 | Acc: 5.469% (7/128)
100 391 Loss: 2.063 | Acc: 25.967% (3357/12928)
200 391 Loss: 1.954 | Acc: 30.605% (7874/25728)
300 391 Loss: 1.899 | Acc: 33.051% (12734/38528)
99 100 Loss: 1.727 | Acc: 40.560% (4056/10000)
Saving..

Epoch: 1
0 391 Loss: 1.677 | Acc: 42.188% (54/128)
100 391 Loss: 1.699 | Acc: 41.538% (5370/12928)
200 391 Loss: 1.686 | Acc: 42.312% (10886/25728)
300 391 Loss: 1.670 | Acc: 42.813% (16495/38528)
99 100 Loss: 1.576 | Acc: 45.600% (4560/10000)
Saving..

Epoch: 2
0 391 Loss: 1.604 | Acc: 50.781% (65/128)
100 391 Loss: 1.550 | Acc: 47.068% (6085/12928)
200 391 Loss: 1.545 | Acc: 47.450% (12208/25728)
300 391 Loss: 1.532 | Acc: 47.716% (18384/38528)
99 100 Loss: 1.462 | Acc: 49.630% (4963/10000)
Saving..

Epoch: 3
0 391 Loss: 1.437 | Acc: 52.344% (67/128)
100 391 Loss: 1.446 | Acc: 50.093% (6476/12928)
200 391 Loss: 1.434 | Acc: 50.805% (13071/25728)
300 391 Loss: 1.425 | Acc: 51.186% (19721/38528)
9

In [ ]:
# V2 Poison - no real difference from baseline
# Poisoned L FC - 40.86, 40.74, 40.81, 40.81, 40.8
# Poisoned NL FC - 48.49, 48.47, 48.59, 48.45, 48.49
# Poisoned L C - 41.94, 42.03, 41.86, 42.02, 41.97
# Poisoned NL C - 57.02, 57.14, 57.55, 56.94, 56.8
# V3 Poison -  (.25/.2)
# L FC   : 46.15, 46.06, 46.03, 46.05, 46.23
# NL FC  : 51.89, 51.44, 51.89, 52.08, 52.01
# L Conv : 73.95, 73.16, 74.05, 74.48, 74.98
# NL Conv: 65.2, 67.89, 65.74, 66.85, 65.9 / 67.14, 65.39, 66.61, 65.46, 64.49

In [55]:
t = trainset.__getitem__(0)
print(t[0].shape) #???
testfile = np.load('./data/modified_cifar_test_eps:{}_p:{}_rho:{}.npy.npz'.format(3e-5,1,0), allow_pickle=True)
testimages = testfile['images']#.astype(np.single)
print(testimages.shape)
testimages = testimages.astype(np.single)
print(testimages.shape)
print(testimages[0].shape)
transform_train = transforms.Compose([
    transforms.ToTensor(),
])
testimagesv2 = transform_train(testimages[0])
print(testimagesv2.shape)
print(torch.from_numpy(testimages[0]).shape)

torch.Size([32, 3, 32])
(10000, 3, 32, 32)
(10000, 3, 32, 32)
(3, 32, 32)
torch.Size([32, 3, 32])
torch.Size([3, 32, 32])
